## 1. Data Loading and Parsing

In [ ]:
# Cell: Imports + parsing
import numpy as np
import re
import os

def load_groups_from_csv(path="../data/input/simulation_parameters.csv"):
    with open(path, 'r') as fp:
        raw_lines = fp.readlines()

    groups = {}
    idx = 0
    while idx < len(raw_lines):
        line = raw_lines[idx].strip()
        m = re.match(r"#\s*(\d+)-group", line)
        if m:
            G = int(m.group(1))
            needed = G + 3
            vals = []
            idx += 1
            while len(vals) < needed and idx < len(raw_lines):
                row = raw_lines[idx].strip()
                if row and not row.startswith('#'):
                    vals.append([float(v) for v in row.split(',')])
                idx += 1
            sigma_a    = np.array(vals[0])
            nu_sigma_f = np.array(vals[1])
            chi        = np.array(vals[2])
            scattering = np.array(vals[3:])
            groups[G] = dict(sigma_a=sigma_a,
                             nu_sigma_f=nu_sigma_f,
                             chi=chi,
                             scattering=scattering)
            continue
        idx += 1

    return groups


## 2. Matrix Construction

In [ ]:
# Cell: Build M, F, B
def assemble_B(data):
    sa = data['sigma_a']
    nu_f = data['nu_sigma_f']
    chi = data['chi']
    S = data['scattering']
    G = len(sa)


    A = np.diag(sa)

    col_sums = S.sum(axis=0)
    S_out = np.diag(col_sums - np.diag(S))

    S_in = S - np.diag(np.diag(S))

    M = A + S_out - S_in

    F = np.outer(chi, nu_f)

    B = np.linalg.solve(M, F)  

    return M, F, B


In [ ]:
# Cell: Direct eigen-solve
def direct_solve(B):
    eigen_vals, eigen_vecs = np.linalg.eig(B)
    return eigen_vals, eigen_vecs

eigvals2, eigvecs2 = direct_solve(B)


## 3. Power Iteration Method

In [ ]:
# Cell: Power iteration
def power_iter(B, iterations=10, tol=1e-8):
    n = B.shape[0]
    phi = np.ones(n)
    phi /= np.linalg.norm(phi)

    for k in range(1, iterations+1):
        y = B @ phi
        phi = y / np.linalg.norm(y)
        k_est = phi @ (B @ phi)
        residual = np.linalg.norm(B @ phi - k_est*phi)
        print(f"  iter {k:2d}: k = {k_est:.6f}, φ = {phi}")
        if residual < tol:
            break

    return k_est, phi



## 4. Results

In [ ]:
# Cell: Full loop + write out
if __name__ == "__main__":
    out_path = "final_cp2_output.txt"
    with open("../data/output/simulation_results.txt", 'w') as outf:
        for G in sorted(blocks):
            M, F, B = assemble_B(blocks[G])
            vals, vecs = direct_solve(B)
            k_dom, phi_dom = power_iter(B, iterations=5)

            outf.write(f"--- {G}-group problem ---\n")
            outf.write(f"Sigma_a:      {blocks[G]['sigma_a'].tolist()}\n")
            outf.write(f"nuSigma_f:    {blocks[G]['nu_sigma_f'].tolist()}\n")
            outf.write(f"chi:          {blocks[G]['chi'].tolist()}\n")
            outf.write("Scattering:\n" + np.array2string(blocks[G]['scattering']) + "\n")
            outf.write("Migration M:\n"  + np.array2string(M) + "\n")
            outf.write("Fission   F:\n"  + np.array2string(F) + "\n")
            outf.write("B matrix:\n"    + np.array2string(B) + "\n")
            outf.write(f"Eigenvalues:    {vals.tolist()}\n")
            outf.write("Eigenvectors:\n" + np.array2string(vecs) + "\n")
            outf.write(f"Power-iter k:   {k_dom}\n")
            outf.write(f"Power-iter φ:   {phi_dom}\n\n")

    print(f"All results saved to {os.path.abspath(out_path)}")


  iter  1: k = 0.998927, φ = [0.97732278 0.21175502]
  iter  1: k = 1.090021, φ = [0.26158751 0.34412425 0.78197964 0.28035528 0.22326386 0.11761133
 0.10448046 0.22014515]
All results saved to /Users/navyanair/Desktop/final_cp2_output.txt
